In [1]:
import pandas as pd
import numpy as np
import re
import subprocess
from pathlib import Path 
import io
import dvc.api, dvc.repo

In [2]:
url = "https://github.com/mostafa-fallaha/data-drift-simulation"
data = dvc.api.read("new_data/Google-Playstore.csv", encoding='utf-8', repo=url)
df = pd.read_csv(io.StringIO(data))
df.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Free,Price,Size_M,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice,Released_Year,Released_Month,Days_Between,Daily_Avg_Installs
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,15,True,0.0,10.0,2020-02-26,2020-02-26,Everyone,False,False,False,2020,2,0,15.0
1,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,58,True,0.0,3.7,2019-08-09,2019-08-19,Everyone,False,False,False,2019,8,10,5.8
2,unlimited 4G data prank free app,getfreedata.superfatiza.unlimitedjiodataprank,Libraries & Demo,4.5,12.0,2567,True,0.0,2.5,2019-09-23,2019-09-27,Everyone,True,False,False,2019,9,4,641.8
3,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2.0,39.0,702,True,0.0,16.0,2019-06-21,2019-06-21,Everyone,False,False,False,2019,6,0,702.0
4,Dodge The Cars!,com.MrScratchEnterprises.CarDogeGame,Racing,4.9,55.0,329,True,0.0,51.0,2020-07-30,2020-07-30,Everyone,False,False,False,2020,7,0,329.0


In [3]:
df.shape

(600000, 19)

In [4]:
df.columns

Index(['App Name', 'App Id', 'Category', 'Rating', 'Rating Count', 'Installs',
       'Free', 'Price', 'Size_M', 'Released', 'Last Updated', 'Content Rating',
       'Ad Supported', 'In App Purchases', 'Editors Choice', 'Released_Year',
       'Released_Month', 'Days_Between', 'Daily_Avg_Installs'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   App Name            600000 non-null  object 
 1   App Id              600000 non-null  object 
 2   Category            600000 non-null  object 
 3   Rating              600000 non-null  float64
 4   Rating Count        600000 non-null  float64
 5   Installs            600000 non-null  int64  
 6   Free                600000 non-null  bool   
 7   Price               600000 non-null  float64
 8   Size_M              600000 non-null  float64
 9   Released            600000 non-null  object 
 10  Last Updated        600000 non-null  object 
 11  Content Rating      600000 non-null  object 
 12  Ad Supported        600000 non-null  bool   
 13  In App Purchases    600000 non-null  bool   
 14  Editors Choice      600000 non-null  bool   
 15  Released_Year       600000 non-nul

In [6]:
df.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Free,Price,Size_M,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice,Released_Year,Released_Month,Days_Between,Daily_Avg_Installs
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,15,True,0.0,10.0,2020-02-26,2020-02-26,Everyone,False,False,False,2020,2,0,15.0
1,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,58,True,0.0,3.7,2019-08-09,2019-08-19,Everyone,False,False,False,2019,8,10,5.8
2,unlimited 4G data prank free app,getfreedata.superfatiza.unlimitedjiodataprank,Libraries & Demo,4.5,12.0,2567,True,0.0,2.5,2019-09-23,2019-09-27,Everyone,True,False,False,2019,9,4,641.8
3,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2.0,39.0,702,True,0.0,16.0,2019-06-21,2019-06-21,Everyone,False,False,False,2019,6,0,702.0
4,Dodge The Cars!,com.MrScratchEnterprises.CarDogeGame,Racing,4.9,55.0,329,True,0.0,51.0,2020-07-30,2020-07-30,Everyone,False,False,False,2020,7,0,329.0


In [9]:
df = df[:90000]

In [10]:
filepath = Path('Google-Playstore.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath, index=False)